In [1]:
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, LSTM, Dense,Embedding,RepeatVector,Bidirectional
from keras.models import Model
import re
import numpy as np
from scipy import stats
# import demoji
import tokenizer
from nltk.corpus import stopwords
stopwords_list = list(set(stopwords.words('english')))


Using TensorFlow backend.


In [2]:
domain1_path = "./dEFEND/gossipcop_content_no_ignore.tsv"
domain2_path = "./dEFEND/politifact_content_no_ignore.tsv"

In [3]:
domain1_frame = pd.read_csv(domain1_path,delimiter="\t").set_index('id')
domain2_frame = pd.read_csv(domain2_path,delimiter="\t").set_index('id')

### Counts for each label for the 2 domain

#### First Domain

In [394]:
domain1_frame

,label,content
id,,
gossipcop-9096198130,1,Sarah Jessica Parker is getting candid about h...
gossipcop-6982710185,1,Many celebrities have been sharing their thoug...
gossipcop-7887456921,1,He reportedly hasn't seen her in over four yea...
gossipcop-1594778479,1,The fashion crowd is speaking out about Kim Ka...
gossipcop-8172018375,1,What term do you want to search? Search with g...
...,...,...
gossipcop-854842,0,Aisha Tyler‘s divorce from Jeffrey Tietjens ha...
gossipcop-843491,0,All four of Queen Elizabeth and Prince Philip'...
gossipcop-897778,0,Theresa Caputo is adjusting to her new life af...


In [395]:
domain1_frame.groupby(['label'])[['label']].count()

,label
label,
0,3586
1,2230


#### Second Domain

In [396]:
domain2_frame.groupby(['label'])[['label']].count()

,label
label,
0,145
1,270


### Cleaning the Text

In [397]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[()\"_#/@;*%:{}<>`+=~|.!?,'$-\[\]]", "", text)
    text = re.sub(r"[0-9]", "", text)
    
#     for words in stopwords_list:
#         text = re.sub(r"\b{}\b".format(words),"",text)
    
    return text

In [398]:
def tagger(decoder_input_sentence):
    start = "<BOS> "
    end = " <EOS>"
    final_target = [start + text + end for text in decoder_input_sentence] 
    return final_target

In [399]:
# def tokenizer(text_lists):
#     return [line.split(" ") for line in text_lists]

In [484]:
def create_vocab(text_lists):
    tokenizer = Tokenizer(oov_token="<UNK>")
    tokenizer.fit_on_texts(text_lists)
    
    dictionary = tokenizer.word_index
    word2idx = {}
    idx2word = {}
    for k,v in dictionary.items():
        word2idx[k]=v
        idx2word[v]=k
    
    return word2idx,idx2word,tokenizer

In [573]:
encoder_inputs = domain1_frame['content'].apply(lambda x: clean_text(x)).values.tolist()+ domain2_frame['content'].apply(lambda x: clean_text(x)).values.tolist()
# encoder_inputs = 
decoder_inputs = tagger(encoder_inputs[:])


In [574]:
word2idx,idx2word,tokenizer = create_vocab(encoder_inputs)

### Due to ambiguity with regards to Keras Tokenizer num_words, below is a good enough fix, though it changes the tokenizer word_index outside of the class

In [575]:
num_words = 30000

sorted_by_word_count = sorted(tokenizer.word_counts.items(), key=lambda kv: kv[1], reverse=True)
tokenizer.word_index = {}
word2idx = {}
idx2word = {}
i = 0
for word,count in sorted_by_word_count:
    if i == num_words:
        break

    tokenizer.word_index[word] = i + 1    # <= because tokenizer is 1 indexed
    word2idx[word] = i+1
    idx2word[i+1]=word
    i += 1





In [576]:
tokenizer.word_index[tokenizer.oov_token] = num_words+1
word2idx[tokenizer.oov_token] = num_words+1
idx2word[num_words+1]=tokenizer.oov_token

In [577]:
# idx = len(tokenizer.word_index)+1
# tokenizer.word_index["<BOS>"] = idx
# word2idx["<BOS>"] = idx
# idx2word[idx] = "<BOS>"

# idx = len(tokenizer.word_index)+1
# tokenizer.word_index["<EOS>"] = idx
# word2idx["<EOS>"] = idx
# idx2word[idx] = "<EOS>"

In [578]:
word2idx["<UNK>"]

30001

In [579]:

t_encoder_inputs = tokenizer.texts_to_sequences(encoder_inputs)
t_decoder_inputs = tokenizer.texts_to_sequences(decoder_inputs)


# 
t_encoder_inputs = pad_sequences(t_encoder_inputs,maxlen=10,padding='post', truncating='post')

# t_decoder_inputs = pad_sequences(t_encoder_inputs,maxlen=100,padding='post', truncating='post')
# t_decoder_inputs = np.insert(t_decoder_inputs,0,word2idx["<BOS>"],axis=1)
# t_decoder_inputs = np.insert(t_decoder_inputs,t_decoder_inputs.shape[1],word2idx["<EOS>"],axis=1)


max_encoder_len = max([len(val) for val in t_encoder_inputs])
max_decoder_len = max([len(val) for val in t_decoder_inputs])


In [580]:
def test_generator(X,Y,batch_size=128,max_len=5):
    
        
    
    for idx in range(0,len(X),batch_size):

        encoder_input = np.zeros((batch_size,max_encoder_len))
#         decoder_input = np.zeros((batch_size,max_decoder_len))
        decoder_target = np.zeros((batch_size,max_encoder_len,len(word2idx)+1))
        for j,input_seq in enumerate(X[idx:idx+batch_size]):
            for i,word_idx in enumerate(input_seq):
                encoder_input[j,i]= word_idx
                decoder_target[j,i,word_idx] = 1
           
        yield [encoder_input,decoder_target]

        

In [581]:
def all_data_generator(X,Y):
    encoder_input = np.zeros((len(X),max_encoder_len))
    decoder_input = np.zeros((len(X),max_decoder_len))
    decoder_target = np.zeros((len(X),max_encoder_len,len(word2idx)+1)) ## Extra index for padding, word2idx is 1 indexed
    for j,(input_seq,target_seq) in enumerate(zip(X,Y)):
        for i,word_idx in enumerate(input_seq):
            encoder_input[j,i]= word_idx
            decoder_target[j,i,word_idx] = 1


    return [encoder_input,decoder_target]

In [582]:
generator = test_generator(t_encoder_inputs,t_encoder_inputs,batch_size=128)

In [583]:
vocab_len= len(word2idx)

In [584]:
test_enc,test_dec = next(generator)

In [585]:
test_enc[0]

array([ 952., 1064., 2305.,    8.,  269., 4031.,   35.,   13.,  136.,
         12.])

### Loading Glove Word Vectors

In [388]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

KeyboardInterrupt: 

In [477]:
embedding_matrix = np.zeros((vocab_len+1, 100))
for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i+1] = embedding_vector

In [478]:
embedding_matrix.shape

(30002, 100)

### Building the Autoencoder

In [586]:
inputs = Input(shape=(max_encoder_len,),name="encoder_inputs")

encoder_embedding = Embedding(vocab_len+1,150,trainable=True,input_length=max_encoder_len,mask_zero=False,name="encoder_embedding")

encoder_inputs = encoder_embedding(inputs)

# encoder_inputs = LSTM(64,return_sequences=True)(encoder_inputs)
encoder = LSTM(64,return_state=True)

# encoder = Bidirectional(LSTM(64, 
#                     return_state=True, 
#                     name = 'encoder'))

# encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# # We discard `encoder_outputs` and only keep the states.

# encoder_states = [forward_h, backward_h]
encoder_states = [state_h,state_c]

# # Set up the decoder, using `encoder_states` as initial state.



decoder_inputs = RepeatVector(max_encoder_len)(encoder_outputs)

# # We set up our decoder to return full output sequences,
# # and to return internal states as well. We don't use the
# # return states in the training model, but we will use them in inference.

decoder_lstm = LSTM(64, 
                         return_state=True,
                        return_sequences=True,
                         name = 'decoder_lstm')


# # The inital_state call argument, specifying the initial state(s) of a RNN. 
# # This is used to pass the encoder states to the decoder as initial states.
# # Basically making the first memory of the decoder the encoded semantics
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)
# decoder_outputs = LSTM(128,return_sequences=True)(decoder_outputs)

# decoder_outputs = Dense(64,activation="tanh")(decoder_outputs)
decoder_dense = Dense(vocab_len+1, 
                      activation='softmax', 
                      name = 'decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

# # Define the model that will turn
# # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model(inputs,decoder_outputs)

In [587]:
model.summary()

Model: "model_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (None, 10)           0                                            
__________________________________________________________________________________________________
encoder_embedding (Embedding)   (None, 10, 150)      4500300     encoder_inputs[0][0]             
__________________________________________________________________________________________________
lstm_21 (LSTM)                  [(None, 64), (None,  55040       encoder_embedding[0][0]          
__________________________________________________________________________________________________
repeat_vector_21 (RepeatVector) (None, 10, 64)       0           lstm_21[0][0]                    
___________________________________________________________________________________________

In [597]:
generator = test_generator(t_encoder_inputs,t_decoder_inputs,batch_size=256)

In [598]:
# [encoder_input_data,decoder_input_data],decoder_target_data = next(generator)
encoder_input_data,decoder_target_data = next(generator)

In [596]:
encoder_input_data,decoder_target_data = all_data_generator(t_encoder_inputs,t_decoder_inputs)

In [599]:
encoder_input_data.shape

(256, 10)

In [600]:
encoder_input_data[127]

array([  61.,  248.,  198., 2020., 9961.,   43.,   15., 1232.,   19.,
        848.])

In [601]:
# [encoder_input_data,decoder_input_data],decoder_target_data = all_data_generator(t_encoder_inputs,t_decoder_inputs)
# encoder_input_data,decoder_target_data = all_data_generator(t_encoder_inputs,t_decoder_inputs)

In [775]:
# decoder_target_data.shape

### Training the Model

In [602]:
encoder_input_data[0]

array([ 952., 1064., 2305.,    8.,  269., 4031.,   35.,   13.,  136.,
         12.])

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
history = model.fit(encoder_input_data, 
                    decoder_target_data,
                    batch_size=128,
                    epochs=40)
#                     validation_split=0.2)

/Users/arnav1712/Desktop/University_Stuff/SMM/Cross-Domain-Fake-News-Detection/venv/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/40
256/256 [==============================] - 5s 21ms/step - loss: 10.3070
Epoch 2/40
256/256 [==============================] - 4s 15ms/step - loss: 10.2923
Epoch 3/40
256/256 [==============================] - 4s 14ms/step - loss: 10.2194
Epoch 4/40
256/256 [==============================] - 4s 14ms/step - loss: 9.8061
Epoch 5/40
256/256 [==============================] - 4s 16ms/step - loss: 9.1049
Epoch 6/40
256/256 [==============================] - 4s 15ms/step - loss: 8.5505
Epoch 7/40
256/256 [==============================] - 4s 14ms/step - loss: 8.1496
Epoch 8/40
256/256 [==============================] - 4s 14ms/step - loss: 7.8484
Epoch 9/40
256/256 [==============================] - 4s 14ms/step - loss: 7.6136
Epoch 10/40
256/256 [==============================] - 4s 14ms/step - loss: 7.4246
Epoch 11/40
256/256 [==============================] - 4s 15ms/step - loss: 7.2700
Epoch 12/40
256/256 [==============================] - 4s 14ms/step - loss: 7.1399
Epoch 13/4

### Testing the Model

In [569]:
for idx in encoder_input_data[0]:
    print(idx2word[idx],end=" ")

jennifer lawrence margot robbie more teach drinking <UNK> from around 

In [570]:
# model.compile('rmsprop', 'mse')
output_array = model.predict([encoder_input_data])

In [571]:
for idx in output_array[1]:
    lookup = np.argmax(idx)
#     print(lookup)
    if lookup==0:
        break
    else:
        print(idx2word[lookup],end=" ")

<UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> 

In [376]:
for idx in output_array[100]:
    print(idx[np.argmax(20)])
    break
    lookup = np.argmax(idx)
#     print(lookup)
    if lookup==0:
        break
    else:
        print(idx2word[lookup],end=" ")

0.00092815526
